### Initial imports

In [1]:
import os
import glob
import numpy as np
import scipy.io
import scipy.special
import warnings
from datetime import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import h5py
from netCDF4 import Dataset 

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import IPython.display
from IPython.display import display, clear_output

warnings.filterwarnings('ignore', category=DeprecationWarning) 

### Some initial track/date info - will need to be changed for NISAR cal/val

In [2]:
tracks  = ['31604','31606']
dates   = ['20120617','20120619','20120622','20120623','20120625','20120627','20120629','20120703','20120705','20120708','20120710','20120713','20120714','20120717']

datetimes = [dt.strptime(date,'%Y%m%d').date() for date in dates]


### Names of retrievals provided by Narendra Das. Used in plot labels.  

In [3]:
mod1 = {'User':'Narendra','Modname':'',
        'Pol':'HH',
        'Path':'./Retrievals/'}
mod2 = {'User':'Narendra','Modname':'',
        'Pol':'VV',
        'Path':'./Retrievals/'}
mod3 = {'User':'Narendra','Modname':'',
        'Pol':'HHVV',
        'Path':'./Retrievals/'}

models = {}
for key in mod1.keys():
    models.setdefault(key,[]).append(mod1[key])        
    models.setdefault(key,[]).append(mod2[key])          
    models.setdefault(key,[]).append(mod3[key]) 

nummods = len(models['User'])    

for i in range(nummods):
    models['Modname'][i]=models['User'][i]+models['Pol'][i]+'HV'
  

### Pre-sorted info on in situ observations during SMAPVEX12

Original data was in several xls, csv files.  For now, merged into single hdf5 file, with fields of:

site: id number

insitu_mean: mean of in situ data for field for each date, with weighting as described in the SMAPVEX12 exercise documentation

insitu_count: number of dates with in situ observations

insitu_all: full set of in situ observations on each day (16 at most sites, 1 in forests) Only used in plotting to show spread of data

In [4]:
insitu_filename = 'fields_insitu.h5'

allfields     = h5py.File(insitu_filename, 'r')
f             = list(allfields.keys())
cropcodenames = {'Grassland':110,
                 'Forage':122,
                 'Oats':136,
                 'Wheat':146,
                 'Corn':147,
                 'CanolaRapeseed':153,
                 'Sunflowers':157,
                 'Soybeans':158,
                 'Broadleaf':220}
cropnames=list(cropcodenames.keys())

### Define the two main matrices where we will store all the data and metrics for each field, and for the crops as a whole.

In [5]:
fields = np.zeros(len(f), dtype = [('site', int),
                                  ('insitu_mean', float , (len(dates),1)),
                                  ('insitu_count', int),
                                  ('insitu_all', float , (len(dates),16)),
                                  ('EASEX',int,(16,1)),
                                  ('EASEY',int,(16,1)),
                                  ('EASEXc',int),
                                  ('EASEYc',int),
                                  ('crop',int),
                                  ('retrievals',float,(nummods,len(tracks),len(dates))), #retrievals
                                  ('ret_count',float,(nummods,len(tracks))),             #number of days with retrievals for each field/model/track
                                  ('res',float,(nummods,len(tracks),len(dates))),        #retrievals-in situ
                                  ('res_count',float,(nummods,len(tracks))),             #number of days with ret AND in situ for each field/model/track
                                  ('res_rmse',float,(nummods,len(tracks))),              #overall rmse
                                  ('res_bias',float,(nummods,len(tracks))),              #bias between retrievals and in situ
                                  ('res_ubrmse',float,(nummods,len(tracks))),            #ub-rmse
                                  ('res_stddev',float,(nummods,len(tracks))),            #std dev of residual
                                  ('res_stddev_stddev',float,(nummods,len(tracks)))])    #std dev of std dev of residual


crops = np.zeros(len(cropnames), dtype = [('cropcode',int),
                                          ('fields',bool,(len(f),1)),
                                          ('numfields',int),
                                          ('rmse',float,(nummods,len(tracks))),
                                          ('rmse_std',float,(nummods,len(tracks))),
                                          ('ubrmse',float,(nummods,len(tracks))),
                                          ('ubrmse_std',float,(nummods,len(tracks))),
                                          ('stddev_uw',float,(nummods,len(tracks))),
                                          ('stddev_wgt',float,(nummods,len(tracks))),
                                          ('stddev_stddev_uw',float,(nummods,len(tracks))),
                                          ('stddev_stddev_wgt',float,(nummods,len(tracks)))])
                              
### Sort the values from our initial allfields hdf5 import into the "fields" ndarray
sites=np.empty(len(f))
for i in range(len(f)):
    siteval=allfields[f[i]]['site'][:]
    sites[i]=siteval[0]
    
    for dset in allfields[f[i]].keys():      
        arr = allfields[f[i]][dset][:] # adding [:] returns a numpy array
        fields[i][dset] = arr
     
sortsiteid=np.argsort(sites)
fields = fields[sortsiteid]

### Make list containing crop names for each field, for use in plotting/pulldown menus.
fieldcrops = list()
for i in range(len(f)):
    if fields[i]['crop'] > 0:
        fieldcrops.append(list(cropcodenames.keys())[list(cropcodenames.values()).index(fields[i]['crop'])])
    else:
        fieldcrops.append('Unknown')
        

### For each retrieval type, pull out the correct values for each field and store them.  Here we are doing it by model, track, and date, so a 3D array for each field.

In [6]:
for k in range(nummods):
    for j in range(len(tracks)):
        nc_f   = models['Path'][k]+'SMAPVEX12_SM200m_UAVSAR_L'  + tracks[j]+ '_' + models['Pol'][k].lower() +'_hv.nc'
        nc_fid = Dataset(nc_f, 'r') 
        dvars  = list(nc_fid.variables)
        grow   = nc_fid.variables['EASE2_row200m'][:]
        gcol   = nc_fid.variables['EASE2_col200m'][:]
      
        id=np.empty(len(f),dtype='int')
        jd=np.empty(len(f),dtype='int')
        for i in range(len(fields)):
            id[i]     = np.argwhere(gcol==fields[i]['EASEXc'])[0][0]
            jd[i]     = np.argwhere(grow==fields[i]['EASEYc'])[0][0]

        for i in range(len(dates)):
            tmpname='SM200m_'+dates[i]
            found = dvars.count(tmpname)
            if found > 0:
                data=nc_fid.variables[tmpname][:]
                data[data<=0] = np.nan
        
                for h in range(0,len(f)):
                    ret=data[id[h],jd[h]]
                    fields[h]['retrievals'][k,j,i]=ret
            
            else:
                for h in range(0,len(fields)):        
                    fields[h]['retrievals'][k,j,i]=np.nan
        
        nc_fid.close()

### Calculate residual between retrievals and in situ data, bias, ubrmse, rmse and the std. deviation of the residual.  

Estimate the std. dev of the std. dev as well, for weighting of individual fields later on. References:         

CR Rao (1973) Linear Statistical Inference and its Applications 2nd Ed, John Wiley & Sons, NY

https://stats.stackexchange.com/questions/11707/why-is-sample-standard-deviation-a-biased-estimator-of-sigma

In [7]:
for i in range(len(fields)):
    for j in range(nummods):
        for k in range(len(tracks)):
            rets       = np.reshape(fields[i]['retrievals'][j,k,:],(len(dates),1))
            dat        = fields[i]['insitu_mean'][:]
            res        = rets-fields[i]['insitu_mean'][:]
            ret_count  = sum(np.isfinite(rets))
            res_count  = sum(np.isfinite(res))

            if res_count>1:
                bias       = np.nanmean(res)
                ubres      = res-bias

                res_rmse   = np.sqrt(np.nansum(np.square(res))/res_count)
                res_ubrmse = np.sqrt(np.nansum(np.square(ubres))/res_count)

                if res_count>1:
                    g1                = scipy.special.gamma((res_count-1)/2)
                    g2                = scipy.special.gamma(res_count/2)
                    res_stddev        = np.sqrt(np.nansum(np.square(ubres))/(res_count-1)) 
                    res_stddev_stddev = res_stddev*g1/g2*np.sqrt((res_count-1)/2-np.square(g2/g1))
                else:
                    res_stddev        = np.nan
                    res_stddev_stddev = np.nan
            else:
                bias                  = np.nan
                res_rmse              = np.nan
                res_ubrmse            = np.nan
                res_stddev            = np.nan
                res_stddev_stddev     = np.nan
                    
            fields[i]['ret_count'][j,k]         = ret_count
            fields[i]['res'][j,k,:]             = np.reshape(res,(len(dates),))
            fields[i]['res_count'][j,k]         = res_count
            fields[i]['res_bias'][j,k]          = bias
            fields[i]['res_rmse'][j,k]          = res_rmse
            fields[i]['res_ubrmse'][j,k]        = res_ubrmse
            fields[i]['res_stddev'][j,k]        = res_stddev
            fields[i]['res_stddev_stddev'][j,k] = res_stddev_stddev


### For each crop type, evaluate the mean of the various metrics, and our uncertainty on those metrics, when possible.

In [8]:
for i in range(len(crops)):  
    code                  = cropcodenames[cropnames[i]]
    fieldid               = np.argwhere(fields['crop']==code)
    fieldidb              = np.reshape(fields['crop']==code,(len(f),1))
    nfields               = len(fieldid)
    crops[i]['cropcode']  = cropcodenames[cropnames[i]]
    crops[i]['numfields'] = nfields
    crops[i]['fields']    = fieldidb
    for j in range(nummods):

        for k in range(len(tracks)):            
            rmse          = np.empty((nfields,1))
            ubrmse        = np.empty((nfields,1))
            stddev        = np.empty((nfields,1))
            stddev_stddev = np.empty((nfields,1))
           
            for l in range(nfields):
                id = fieldid[l][0]
                rmse[l]          = fields[id]['res_rmse'][j,k]
                ubrmse[l]        = fields[id]['res_ubrmse'][j,k]
                stddev[l]        = fields[id]['res_stddev'][j,k]
                stddev_stddev[l] = fields[id]['res_stddev_stddev'][j,k]
            
            goodfields           = np.sum(np.isfinite(rmse))     
            if goodfields > 0:
                wgts             = 1. / np.square(stddev_stddev)
                V1               = np.nansum(wgts)
                V2               = np.nansum(np.square(wgts))

                crops[i]['rmse'][j,k]                  = np.nanmean(rmse)
                crops[i]['ubrmse'][j,k]                = np.nanmean(ubrmse)
                crops[i]['stddev_uw'][j,k]             = np.nanmean(stddev)
                crops[i]['stddev_wgt'][j,k]            = np.nansum(stddev*wgts)/V1
                
                if (V1-(V2/V1)) > 0:
                    crops[i]['stddev_stddev_wgt'][j,k] = np.sqrt(np.nansum(wgts*np.square(stddev-np.nanmean(stddev)))/(V1-(V2/V1)));
                else:
                    crops[i]['stddev_stddev_wgt'][j,k] = np.nan
            else:
                crops[i]['rmse'][j,k]                  = np.nan
                crops[i]['ubrmse'][j,k]                = np.nan
                crops[i]['stddev_uw'][j,k]             = np.nan
                crops[i]['stddev_wgt'][j,k]            = np.nan
            
            
            if goodfields > 1:
                crops[i]['rmse_std'][j,k]              = np.nanstd(rmse,ddof=1)
                crops[i]['ubrmse_std'][j,k]            = np.nanstd(ubrmse,ddof=1)
                crops[i]['stddev_stddev_uw'][j,k]      = np.nanstd(stddev,ddof=1)

            else:
                crops[i]['rmse_std'][j,k]              = np.nan
                crops[i]['ubrmse_std'][j,k]            = np.nan
                crops[i]['stddev_stddev_uw'][j,k]      = np.nan


# Plotting

### Below, we define several plotting functions, and include a single example that drives each of them below each set of code.  At the end of the notebook, are tabs with all of the plotting types in one spot.  

In [9]:
def plot_onefield(fieldnum,correctionType):
    insitu_mean = fields[fieldnum]['insitu_mean']
    insitu_all  = fields[fieldnum]['insitu_all']

    fig = plt.figure(figsize=(14,5))
    ax  = fig.subplots(1)
    markers=['o','d']

    for i in range(len(tracks)):
        for j in range(nummods):
            if fields[fieldnum]['res_count'][j,i] > 0:
                ret        = np.reshape(fields[fieldnum]['retrievals'][j,i,:],(len(dates),1))
                if correctionType == 'unbiased':
                    diff       = ret-insitu_mean
                    ret        = ret-np.nanmean(diff,0)                   
                elif correctionType == 'residual':
                    ret        = ret-insitu_mean
                elif correctionType == 'unbiased residual':
                    diff       = ret-insitu_mean
                    ret        = ret-np.nanmean(diff,0)-insitu_mean
                            
                plt.plot(datetimes,ret,':',marker=markers[i],markersize=10,label=models['Modname'][j]+str(tracks[i]))


    if correctionType == 'residual':
        insitu_all  =insitu_all-insitu_mean
        insitu_mean = 0*insitu_mean
    elif correctionType == 'unbiased residual':
        insitu_all  =insitu_all-insitu_mean
        insitu_mean = 0*insitu_mean

    plt.plot(datetimes,insitu_mean,'ko',markersize=18,markerfacecolor='w',label='Mean insitu')
    plt.plot(datetimes,insitu_all,'k.')
    plt.plot(datetimes,insitu_all[:,0],'k.',label='All insitu')#plot one row for label

    plt.title('Field'+str(fields[fieldnum]['site'])+' '+fieldcrops[fieldnum])
    ax.set_ylabel('Volumetric Soil Moisture')
    ax.grid()
    ax.set_xticks(datetimes[0::2])
    ax.legend()

    plt.xlim(datetimes[0]+timedelta(-1),datetimes[-1]+timedelta(1))
    plt.show()

In [10]:
fval1=[]
for i in range(len(fields)):
    fval1.append(str(fields[i]['site'])+':'+fieldcrops[i])

pulldownfield1=widgets.Dropdown(description='Field:',options=fval1)
pulldownplot1=widgets.Dropdown(description='Plottype:',options=['raw','unbiased','residual','unbiased residual'])

button1 = widgets.Button(description='Plot')
out1 = widgets.Output()
def button1_clicked(_):
      with out1:
          clear_output()
          plot_onefield(pulldownfield1.index,pulldownplot1.value)

button1.on_click(button1_clicked)
widgets.VBox([widgets.HBox([pulldownfield1,pulldownplot1,button1]),out1])


In [11]:
def plot_onecrop_byfield(cropnum,modnum,tracknum):
    goodfields=np.argwhere(crops[cropnum]['fields'])[:,0]

    fig = plt.figure(figsize=(14,5))
    ax  = fig.subplots(1)
    ax.grid('on')
    x   = np.arange(1,len(goodfields)+1)
    fieldlabels=[]
    for i in range(len(goodfields)):
        ax.errorbar(x[i],fields[goodfields[i]]['res_stddev'][modnum,tracknum], yerr=fields[goodfields[i]]['res_stddev_stddev'][modnum,tracknum], xerr=None,fmt='.',color='r',ecolor='r',capsize=10,markersize=18)
        ax.plot(x[i],fields[goodfields[i]]['res_ubrmse'][modnum,tracknum],'go') 
        fieldlabels.append('F'+str(fields[goodfields[i]]['site'])+' : '+str(np.int32(fields[goodfields[i]]['res_count'][modnum,tracknum])))

    #plot again for labels
    ax.errorbar(x[i],fields[goodfields[i]]['res_stddev'][modnum,tracknum], yerr=fields[goodfields[i]]['res_stddev_stddev'][modnum,tracknum], xerr=None,fmt='.',color='r',ecolor='r',capsize=10,markersize=18,label=r'$\sigma$')

    ax.hlines(crops[cropnum]['stddev_uw'][modnum,tracknum],0,len(goodfields)+0.5,colors='b')
    ax.hlines(crops[cropnum]['stddev_wgt'][modnum,tracknum],0,len(goodfields)+0.5,colors='k')
    ax.hlines(crops[cropnum]['ubrmse'][modnum,tracknum],0,len(goodfields)+0.5,colors='g')
    ax.errorbar(0.3,crops[cropnum]['stddev_uw'][modnum,tracknum],yerr=crops[cropnum]['stddev_stddev_uw'][modnum,tracknum],fmt='bo',label=r'tot $\sigma$ unwgt')
    ax.errorbar(0.5,crops[cropnum]['stddev_wgt'][modnum,tracknum],yerr=crops[cropnum]['stddev_stddev_wgt'][modnum,tracknum],fmt='ko',label=r'tot $\sigma$ wgt')
    ax.errorbar(0.7,crops[cropnum]['ubrmse'][modnum,tracknum],yerr=crops[cropnum]['ubrmse_std'][modnum,tracknum],fmt='go',label=r'tot $\sigma$ UBRMSE')
    plt.xlim(0,len(goodfields)+0.5)
    ax.set_xticks(x)
    ax.set_xticklabels(fieldlabels)
    ax.set_xlabel('Field# : #obs')
    ax.set_ylabel('Error')
    ax.legend()
    plt.title(cropnames[cropnum]+' T'+str(tracks[tracknum])+', '+models['Modname'][modnum])
    plt.show()

In [12]:
fval2=[]
for i in range(len(crops)):
    fval2.append(cropnames[i]+':'+str(crops[i]['numfields']) + ' fields')

pulldowncrop2  = widgets.Dropdown(description='Crop:',options=fval2)
pulldownmod2   = widgets.Dropdown(description='Model:',options=models['Modname'])
pulldowntrack2 = widgets.Dropdown(description='Track:',options=tracks)

button2 = widgets.Button(description='Plot')
out2 = widgets.Output()
def button2_clicked(_):
      with out2:
          clear_output()
          plot_onecrop_byfield(pulldowncrop2.index,pulldownmod2.index,pulldowntrack2.index)

button2.on_click(button2_clicked)
widgets.VBox([button2,widgets.HBox([pulldowncrop2,pulldownmod2,pulldowntrack2]),out2])

In [13]:
def plot_onecrop(cropnum,correctionType):
    tb1=(-0.15,0.15) #shifts for each track in plotting bars
    tb2=(-0.2,0.2) #shifts for each track in plotting points

    color=cm.rainbow(np.linspace(0,1,len(tracks)))

    fig = plt.figure(figsize=(14,5))
    ax  = fig.subplots(1)
    ax.grid('on')
    x   = np.arange(1,nummods+1)
    
    ax.plot([0.5,nummods+1],[0.06,0.06],'k',label='6%')
    if correctionType == 'Weighted':
        for i in range(len(tracks)):
            ax.bar(x+tb1[i],crops[cropnum]['stddev_wgt'][:,i],width=0.25,color=color[i],label=r'$\sigma_{wgt}$ '+str(tracks[i]))
            ax.errorbar(x+tb1[i], crops[cropnum]['stddev_wgt'][:,i], yerr=crops[cropnum]['stddev_stddev_wgt'][:,i], xerr=None,fmt='none',ecolor='k',capsize=10)
            ax.errorbar(x+tb2[i],crops[cropnum]['rmse'][:,i], yerr=crops[cropnum]['rmse_std'][:,i], xerr=None,fmt='.',color=color[i],ecolor='k',capsize=10,markersize=18,label='RMSE '+str(tracks[i]))
            plt.title(cropnames[cropnum]+', '+str(crops[cropnum]['numfields'])+' fields, wgt error')
    elif correctionType == 'Unweighted':
        for i in range(len(tracks)):
            ax.bar(x+tb1[i],crops[cropnum]['stddev_uw'][:,i],width=0.25,color=color[i],label=r'$\sigma_{unwgt}$ '+str(tracks[i]))
            ax.errorbar(x+tb1[i], crops[cropnum]['stddev_uw'][:,i], yerr=crops[cropnum]['stddev_stddev_uw'][:,i], xerr=None,fmt='none',ecolor='k',capsize=10)
            ax.errorbar(x+tb2[i],crops[cropnum]['rmse'][:,i], yerr=crops[cropnum]['rmse_std'][:,i], xerr=None,fmt='.',color=color[i],ecolor='k',capsize=10,markersize=18,label='RMSE '+str(tracks[i]))
            plt.title(cropnames[cropnum]+', '+str(crops[cropnum]['numfields'])+' fields, unwgt error')
    elif correctionType == 'UBRMSE':
        for i in range(len(tracks)):
            ax.bar(x+tb1[i],crops[cropnum]['ubrmse'][:,i],width=0.25,color=color[i],label='UBRMSE '+str(tracks[i]))
            ax.errorbar(x+tb1[i], crops[cropnum]['ubrmse'][:,i], yerr=crops[cropnum]['ubrmse_std'][:,i], xerr=None,fmt='none',ecolor='k',capsize=10)
            ax.errorbar(x+tb2[i],crops[cropnum]['rmse'][:,i], yerr=crops[cropnum]['rmse_std'][:,i], xerr=None,fmt='.',color=color[i],ecolor='k',capsize=10,markersize=18,label='RMSE '+str(tracks[i]))
            plt.title(cropnames[cropnum]+', '+str(crops[cropnum]['numfields'])+' fields, UBRMSE')

    plt.xlim(0.5,nummods+1)
    ax.set_xticks(x)
    ax.set_xticklabels(models['Modname'])
    ax.set_xlabel('Model')
    ax.set_ylabel('Error')
    ax.legend()
    plt.show()
   

In [14]:
pulldowncrop3=widgets.Dropdown(description='Crop:',options=fval2)
pulldownplot3=widgets.Dropdown(description='Plottype:',options=['Weighted','Unweighted','UBRMSE'])

button3 = widgets.Button(description='Plot')
out3 = widgets.Output()
def button3_clicked(_):
      with out3:
          clear_output()
          plot_onecrop(pulldowncrop3.index,pulldownplot3.value)

button3.on_click(button3_clicked)
widgets.VBox([widgets.HBox([pulldowncrop3,pulldownplot3,button3]),out3])

In [15]:
def plot_allcrops(type):
    thresh=0.06;
    mytitle=type;
    all=np.empty((len(crops),nummods))
    if type == 'RMSE':
        for i in range(len(crops)):
            for j in range(nummods):
                all[i,j]=np.nanmean(crops[i]['rmse'][j,:])
    elif type == 'UBRMSE':
        for i in range(len(crops)):
            for j in range(nummods):
                all[i,j]=np.nanmean(crops[i]['ubrmse'][j,:])
    elif type == 'std dev, weighted':
        mytitle = r'$\sigma$, weighted'
        for i in range(len(crops)):
            for j in range(nummods):
                all[i,j]=np.nanmean(crops[i]['stddev_wgt'][j,:])
    elif type == 'std dev, unweighted':
        mytitle = r'$\sigma$, unweighted'
        for i in range(len(crops)):
            for j in range(nummods):
                all[i,j]=np.nanmean(crops[i]['stddev_uw'][j,:])

    id  = np.where(all<=thresh)
    fig = plt.figure(figsize=(14,5))
    ax  = fig.subplots(1)
    p   = ax.imshow(all,aspect='auto',vmin=0,vmax=0.1)
    if type == 'RMSE':
        p   = ax.imshow(all,aspect='auto',vmin=0,vmax=0.2)
    else:
        p   = ax.imshow(all,aspect='auto',vmin=0,vmax=0.1)
 
    ax.plot(id[1],id[0],'wo')
    ax.set_xticks(range(nummods))
    ax.set_xticklabels(models['Modname'])
    ax.set_xlabel('Model')
    ax.set_yticks(range(len(crops)))
    ax.set_yticklabels(cropnames)
    cbar=plt.colorbar(p)
    cbar.set_label('Error', rotation=270)
    plt.title(mytitle)
    plt.show()  

In [16]:
fval4=[]
for i in range(len(crops)):
    fval4.append(cropnames[i]+':'+str(crops[i]['numfields']) + ' fields')

pulldownplot4=widgets.Dropdown(description='Plottype:',options=['RMSE','UBRMSE','std dev, weighted','std dev, unweighted'])

button4 = widgets.Button(description='Plot')
out4 = widgets.Output()
def button4_clicked(_):
      with out4:
          clear_output()
          plot_allcrops(pulldownplot4.value)

button4.on_click(button4_clicked)

widgets.VBox([widgets.HBox([pulldownplot4,button4]),out4])

In [17]:
# Put all plots into tabs
tab = widgets.Tab()
tab.children = [widgets.VBox([widgets.HBox([pulldownfield1,pulldownplot1,button1]),out1]), 
                widgets.VBox([button2,widgets.HBox([pulldowncrop2,pulldownmod2,pulldowntrack2]),out2]),
                widgets.VBox([widgets.HBox([pulldowncrop3,pulldownplot3,button3]),out3]),
                widgets.VBox([widgets.HBox([pulldownplot4,button4]),out4])]
tab.set_title(0, 'Plot one field')
tab.set_title(1, 'Plot one crop, by field')
tab.set_title(2, 'Plot one crop')
tab.set_title(3, 'Plot all crops')
tab